In [4]:
#this is the initial import of the diff files, loading them into pandas dataframes
import pandas as pd
import os
import numpy as np
from csv_diff import load_csv, compare
folderpath = r"D:\jayData\2020_12_diffs" # make sure to put the 'r' in front
filepaths  = [os.path.join(folderpath, name) for name in os.listdir(folderpath)]
all_files = []
print(filepaths)
header = ['tax_id', 'GeneID', 'Symbol', 'LocusTag', 'Synonyms', 'dbXrefs', 'chromosome', 'map_location', 'description', 'type_of_gene', 'Symbol_from_nomenclature_authority', 'Full_name_from_nomenclature_authority', 'Nomenclature_status', 'Other_designations', 'Modification_date', 'Feature_type']
data = pd.read_csv("D:/jayData/2020_12_diffs/20201201_20201202.diff", delimiter = "\t", skiprows = 3)
data.columns = header
dfList = []
for i in range(22):
    globals()['df%s' % i] = pd.read_csv(filepaths[i], delimiter = "\t", skiprows = 3)
    globals()['df%s' % i].columns = header
    dfList.append(globals()['df%s' % i])

['D:\\jayData\\2020_12_diffs\\20201201_20201202.diff', 'D:\\jayData\\2020_12_diffs\\20201202_20201203.diff', 'D:\\jayData\\2020_12_diffs\\20201203_20201204.diff', 'D:\\jayData\\2020_12_diffs\\20201204_20201205.diff', 'D:\\jayData\\2020_12_diffs\\20201205_20201206.diff', 'D:\\jayData\\2020_12_diffs\\20201206_20201208.diff', 'D:\\jayData\\2020_12_diffs\\20201208_20201209.diff', 'D:\\jayData\\2020_12_diffs\\20201209_20201210.diff', 'D:\\jayData\\2020_12_diffs\\20201210_20201211.diff', 'D:\\jayData\\2020_12_diffs\\20201211_20201212.diff', 'D:\\jayData\\2020_12_diffs\\20201212_20201214.diff', 'D:\\jayData\\2020_12_diffs\\20201214_20201215.diff', 'D:\\jayData\\2020_12_diffs\\20201215_20201216.diff', 'D:\\jayData\\2020_12_diffs\\20201216_20201217.diff', 'D:\\jayData\\2020_12_diffs\\20201217_20201220.diff', 'D:\\jayData\\2020_12_diffs\\20201220_20201223.diff', 'D:\\jayData\\2020_12_diffs\\20201223_20201224.diff', 'D:\\jayData\\2020_12_diffs\\20201224_20201225.diff', 'D:\\jayData\\2020_12_diffs

c:\users\atwil\appdata\local\programs\python\python38\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
c:\users\atwil\appdata\local\programs\python\python38\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (0,6) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [5]:
n = 0
for df in dfList:
    df = df[df['tax_id'].str.contains('9606', na = False)]
    df.to_csv('d:/jayData/2020_12_human_diffs/diff%s.csv' % n, index = False, sep = '\t')
    n += 1

In [6]:
newDfList = []
n = 0

for df in dfList:
    print(n)
    diff_symbols = []
    df = df[['tax_id', 'GeneID', 'Synonyms', 'dbXrefs', 'chromosome', 'Modification_date']]
    df = df[df['tax_id'].str.contains('9606', na = False)]
    for i, row in df.iterrows():
        diff_symbols.append(df.at[i, 'tax_id'][0])
#         if df.at[i, 'tax_id'][0] == ' ':
#             df.drop(i, inplace = True)

    synonym_count = []
    dbXrefs_count = []
    has_hgnc = []
    has_mim = []
    has_ensembl = []
    for i, row in df.iterrows():
        if isinstance(df.at[i, 'Synonyms'], str):
            if df.at[i, 'Synonyms'] == '-':
                synonym_count.append(0)
            else:
                synonym_count.append(str(df.at[i, 'Synonyms'].count('|') + 1))
        if isinstance(df.at[i, 'Synonyms'], float):
            synonym_count.append(0)
    
    for i, row in df.iterrows():
        val = df.at[i, 'dbXrefs']
        if isinstance(val, str):
            if val == '-':
                dbXrefs_count.append(0)
                has_hgnc.append(False)
                has_mim.append(False)
                has_ensembl.append(False)
            else:
                dbXrefs_count.append(str(val.count('|') + 1))
                has_hgnc.append('HGNC' in val)
                has_mim.append('MIM' in val)
                has_ensembl.append('Ensembl' in val)
        if isinstance(val, float):
            dbXrefs_count.append(0)
            has_hgnc.append(False)
            has_mim.append(False)
            has_ensembl.append(False)
    
    indelupdateCols = []
    for i, row in df.iterrows():
#         temp = df[df['tax_id'].str.contains(str(df.at[i, 'GeneID']), na = False)]
        sub = df[df.values == df.at[i, 'GeneID']]
        if sub.shape[0] > 1:
            if sub.iloc[0]['Modification_date'] != sub.iloc[1]['Modification_date']:
                indelupdateCols.append('update - date only')
            else:
                indelupdateCols.append('update')
        if sub.shape[0] <= 1:
            if df.at[i, 'tax_id'][0] == ' ':
                indelupdateCols.append('NA')
            if df.at[i, 'tax_id'][0] == '+':
                indelupdateCols.append('insertion')
            if df.at[i, 'tax_id'][0] == '-':
                indelupdateCols.append('deletion')
            if df.at[i, 'tax_id'][0] == '@':
                indelupdateCols.append('NA')
                
    for i, row in df.iterrows():
        df.at[i, 'tax_id'] = df.at[i,'tax_id'][1:]
        
    df['synonym_count'] = synonym_count
    df['dbXrefs_count'] = dbXrefs_count
    df['has_hgnc'] = has_hgnc
    df['has_mim'] = has_mim
    df['has_ensembl'] = has_ensembl
    df['diff_symbol'] = diff_symbols
    df['insert_delete_or_update'] = indelupdateCols
    newDfList.append(df.copy())
    n += 1

0
1
2
3
4
5
6
7
8
9
10
11


KeyboardInterrupt: 

In [ ]:
sum = 0
for df in newDfList:
    print(df.shape[0])
    sum += df.shape[0]

In [ ]:
print(newDfList[1])

In [ ]:
n = 0
for df in newDfList:
    df.to_csv('d:/jayData/2020_12_human_diff_analyses/diffAnal%s.csv' % n, index = False, sep = '\t')
    n += 1


In [ ]:
datesUpdateList = []
for df in newDfList:
    df = df[~df['insert_delete_or_update'].str.contains('date only', na = False)]
    df = df[~df['insert_delete_or_update'].str.contains('NA', na = False)]

    datesUpdateList.append(df.copy(deep = True))
    print(df)

In [ ]:
#         print(sub)
# for i in range(0, indelTable.shape[0]):
#     try:
#         if indelTable.iloc[i]['GeneID'] == indelTable.iloc[i+1]['GeneID']:
#             indelupdateCols.append('update')
# #             if indelTable.iloc[i]['diff_symbols'] == '-' and indelTable.iloc[i+1]['diff_symbols'] == '+':
# #                 indelupdateCols.append('update')
#         elif indelTable.iloc[i]['GeneID'] == indelTable.iloc[i-1]['GeneID']:
#             indelupdateCols.append('update')
#         elif indelTable.iloc[i]['GeneID'] != indelTable.iloc[i+1]['GeneID'] or indelTable.iloc[i]['GeneID'] != indelTable.iloc[i-1]['GeneID']:
#             if indelTable.iloc[i]['diff_symbols'] == '+':
#                 indelupdateCols.append('insertion')
#             if indelTable.iloc[i]['diff_symbols'] == '-':
#                 indelupdateCols.append('deletion')
#     except(IndexError):
#         continue

In [200]:
keys = indelTable['chromosome'].unique()
chromCountDict = dict.fromkeys(keys, 0)
for i, row in indelTable.iterrows():
    key = indelTable.at[i, 'chromosome']
    try:
        chromCountDict[key] += 1
    except KeyError:
        continue
chromCountList = list(chromCountDict.items())
for i in range(len(chromCountList)- 1):
    for j in range(i + 1, len(chromCountList)):
        if chromCountList[i][1] > chromCountList[j][1]:
            temp = chromCountList[i]
            chromCountList[i] = chromCountList[j]
            chromCountList[j] = temp
    chromCountDict = dict(chromCountList)

In [201]:
print(chromCountDict)

{'MT': 12, 'X|Y': 14, 'Y': 66, nan: 132, '-': 226, '21': 262, '18': 400, '13': 458, '22': 527, '20': 656, '15': 714, '8': 842, '14': 854, '16': 925, '10': 958, 'X': 990, '4': 997, '9': 1016, '5': 1104, '7': 1255, '12': 1289, '6': 1318, '3': 1348, '17': 1438, '11': 1494, '2': 1559, '19': 1561, '1': 2386}


In [205]:
# dbXrefs
keys = indelTable['dbXrefs_count'].unique()
dbXrefsCountDict = dict.fromkeys(keys, 0)
for i, row in indelTable.iterrows():
    key = indelTable.at[i, 'dbXrefs_count']
    try:
        dbXrefsCountDict[key] += 1
    except KeyError:
        continue
dbXrefsCountList = list(dbXrefsCountDict.items())
for i in range(len(dbXrefsCountList) - 1):
    for j in range(i + 1, len(dbXrefsCountList)):
        if dbXrefsCountList[i][1] > dbXrefsCountList[j][1]:
            temp = dbXrefsCountList[i]
            dbXrefsCountList[i] = dbXrefsCountList[j]
            dbXrefsCountList[j] = temp
    dbXrefsCountDict = dict(dbXrefsCountList)


In [206]:
print(dbXrefsCountDict)

{5: 6, 4: 449, 0: 500, 1: 825, 2: 2267, 3: 20754}


In [209]:
# synonyms
keys = indelTable['synonym_count'].unique()
synonymCountDict = dict.fromkeys(keys, 0)
for i, row in indelTable.iterrows():
    key = indelTable.at[i, 'synonym_count']
    try:
        synonymCountDict[key] += 1
    except KeyError:
        continue
synonymCountList = list(synonymCountDict.items())
for i in range(len(synonymCountList) - 1):
    for j in range(i + 1, len(synonymCountList)):
        if synonymCountList[i][1] > synonymCountList[j][1]:
            temp = synonymCountList[i]
            synonymCountList[i] = synonymCountList[j]
            synonymCountList[j] = temp
    synonymCountDict = dict(synonymCountList)

In [210]:
print(synonymCountDict)

{23: 2, 17: 2, 19: 16, 15: 22, 18: 24, 16: 30, 14: 52, 13: 76, 12: 120, 11: 202, 10: 311, 9: 375, 8: 669, 7: 1087, 6: 1437, 5: 2383, 0: 3014, 4: 3301, 1: 3751, 3: 3770, 2: 4157}
